In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import re
import plotly as pl 
import matplotlib.pyplot as plt
from pathlib import Path
import psycopg2
import sys
import boto3
import os
import sqlalchemy

In [10]:
lat_long_data = "https://chocolate-final-project-21.s3.us-east-2.amazonaws.com/country_and_usa_states_lat_long.csv"
df = pd.read_csv(lat_long_data)
#lat_long_df.head(10)



#file_path = Path('country_and_usa_states_lat_long.csv')
#df = pd.read_csv(file_path)
df.head()



,country_code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [3]:
# Store environmental variable
from getpass import getpass
enter_password = getpass('Enter database password')

connection = psycopg2.connect(
    host = "database-chocolate.cafzzay3t2tr.us-east-2.rds.amazonaws.com",
    port = 5432,
    user = 'postgres',
    password = enter_password,
    database = 'postgres'
    )

cursor = connection.cursor()
cursor

Enter database password········


<cursor object at 0x7f8e54857550; closed: 0>

In [ ]:
# Create location table

cursor.execute(""" CREATE TABLE location_table(
country_code text,
latitude float,
longitude float,
country text)""")

connection.commit()

In [ ]:
# Load data into postgres table from csv

with open ('country_and_usa_states_lat_long.csv', 'r') as row:
           next(row) # skip the header row
           cursor.copy_from(row, 'location_table', sep = ',')
                   
connection.commit()

In [8]:
sql = """
SELECT * FROM location_table
"""

df_from_db = pd.read_sql(sql, con=connection)

In [9]:
df_from_db.head(5)

,country_code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [ ]:
# Close connection
cursor.close()

In [ ]:
cursor